# Detection of Chessboard

## Setup

In [ ]:
import numpy as np
import cv2 as cv

from PIL import Image
from IPython.display import display, clear_output

import line_and_point_detect as lp

def show_img(img):
    # convert color from CV2 BGR back to RGB
    image = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    display(Image.fromarray(image))

## Read image

In [ ]:
# filename = "Chessboard_00451_2.png" # sin
# filename = "Chessboard_00451.png" # normal
filename = "Chessboard_0481_0.png" # noise
# filename = "Chessboard_0481.png"
# filename = "Chessboard_0511.png"
# filename = "Chessboard_0541.png"
# filename = "Chessboard_00601.png"
# filename = "Chessboard0631.png"

img = cv.imread(filename)
show_img(img)

## Edge Detection
 
### Convert the image to grayscale

In [ ]:
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
show_img(gray)

### Remove sinusoidal noise

In [ ]:
def average_color(img):
    # get the average color of the image
    return np.sum(img) / (np.shape(img)[0] * np.shape(img)[1])

def remove_sinusoidal_noise(img):
    # Convert the image to a 2D numpy array
    img = np.array(img)

    # Apply the FFT to the image
    f = np.fft.fft2(img)

    # Shift the zero-frequency component to the center
    fshift = np.fft.fftshift(f)

    # To see if components were removed
    removed = False

    # Get the rows and columns of the image
    rows, cols = img.shape

    # Set the noise-removal threshold for the other frequency components
    other_freq_threshold = 1000 * average_color(np.abs(fshift))

    # Zero out the high-frequency components that correspond to noise
    for y in range(rows):
        for x in range(cols):
            if not (y == rows // 2 and x == cols // 2):
                if np.abs(fshift[y][x]) > other_freq_threshold:
                    fshift[y][x] = 0
                    removed = True

    # Shift the zero-frequency component back to the original location
    f_ishift = np.fft.ifftshift(fshift)

    # Apply the inverse FFT to the filtered image
    img_back = np.fft.ifft2(f_ishift)

    # Convert the result back to an 8-bit unsigned integer
    img_back = np.abs(img_back).astype(np.uint8)
    return [img_back, removed]



img_nosin, is_sin_removed = remove_sinusoidal_noise(gray)
show_img(img_nosin)

if is_sin_removed:
    gray = img_nosin


### Laplacian of Gaussian
#### Apply Gaussian filter

In [ ]:
# gaussian = cv.GaussianBlur(gray, (3, 3), 0)
gaussian = cv.GaussianBlur(gray, (15, 15), 0)

show_img(gaussian)

#### Apply [Laplacian of Gaussian filter](https://docs.opencv.org/3.4/d5/db5/tutorial_laplace_operator.html)

In [ ]:
log = cv.Laplacian(gaussian, cv.CV_8U, ksize=5)

show_img(log)

#### Combine the gaussian blurr and the log edges.

The weight is ranging from 0 to 1. A value of 1 means the image will be fully visible in the output, while a value of 0 means it will be fully transparent.

In [ ]:
result = cv.addWeighted(gaussian, 1, log, 0.5, 0)
show_img(result)


### Apply Line detection

In [ ]:
canny = cv.Canny(result, 70, 120)
show_img(canny)

### Lines

In [ ]:
line_tab = []
line_tab = lp.lines_detector(canny, 120)
show_img(lp.draw_lines(img, line_tab)) # Lines

In [ ]:
line_tabP = lp.lines_detector_P(canny, 50, 100, 150)
show_img(lp.draw_lines(img, line_tabP)) # LinesP

### Points

In [ ]:
intersectP = lp.find_all_intersection(img, line_tabP)
img2 = np.copy(img)
#

for point in intersectP:
    cv.circle(img2, (int(point[0]), int(point[1])), 5, (0, 255, 0), 3)

show_img(img2)

In [ ]:
print(len(intersectP), len(line_tab))